In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# 로이터 뉴스 데이터 셋 불러오기
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input,Reshape, Flatten  
from tensorflow.keras.layers import Embedding , LSTM, BatchNormalization, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [2]:
import os
if not os.path.exists("gan_images"):
    os.makedirs("gan_images")

In [3]:
generator = Sequential() 
generator.add(Dense(128*7*7, input_dim=100,\
                    activation=LeakyReLU(0.2))) 
generator.add(BatchNormalization()) 
generator.add(Reshape((7, 7, 128))) 
generator.add(UpSampling2D()) 
generator.add(Conv2D(64, kernel_size=5, padding='same')) 
generator.add(BatchNormalization()) 
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D()) 
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) 


In [4]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
batch_normalization (BatchNo (None, 6272)              25088     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 14, 14, 64)        0

In [5]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

In [6]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)             

In [7]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam') 

In [8]:
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _)=mnist.load_data()
    X_train=X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')
    X_train=(X_train-127.5)/127.5 #절반을 빼고, 그 절반으로 나누면 -1~1사이의 range로 변환됨
    true=np.ones((batch_size,1))
    fake=np.zeros((batch_size,1))
    for i in range(epoch):
        idx=np.random.randint(0, X_train.shape[0], batch_size)
        imgs=X_train[idx]
        d_loss_real=discriminator.train_on_batch(imgs, true) # 데이터를 학습시키는 부분
        #train_on_batch로 학습 (input, target)
        noise=np.random.normal(0,1,(batch_size,100))  # 0과 1사이로 정규분포 노이즈는 100개 발생시킨다
        gen_imgs=generator.predict(noise)  # 가중치와 바이어스가 랜덤하게 되어 있어서 말도 안되는 그림이 나온다
        d_loss_fake=discriminator.train_on_batch(gen_imgs,fake) # 이렇게 나온 놈은 가짜이다.
        #판별자와 생성자의 오차 계산
        d_loss=0.5*np.add(d_loss_real, d_loss_fake)
        g_loss=gan.train_on_batch(noise,true) # discriminator 학습이 일어나지 않는다. 오직 gan 만 일어난다.
        print('epoch:%d' %i, 'd_loss:%.4f' %d_loss, 'g_loss:%.4f' %g_loss)
        if i % saving_interval == 0:  # 소스를 받은거, 중간 세이브 , 실제 액션은 윗부분
              #r, c = 5, 5
              noise = np.random.normal(0, 1, (25, 100))
              gen_imgs = generator.predict(noise)
              # Rescale images 0 - 1
              gen_imgs = 0.5 * gen_imgs + 0.5
              fig, axs = plt.subplots(5, 5)
              count = 0
              for j in range(5):
                  for k in range(5):
                      axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                      axs[j, k].axis('off')
                      count += 1
              fig.savefig("gan_images/gan_mnist_%d.png" % i)

In [9]:
gan_train(4001,32,200)

epoch:0 d_loss:0.7503 g_loss:0.6419
epoch:1 d_loss:0.5240 g_loss:0.3887
epoch:2 d_loss:0.4932 g_loss:0.1363
epoch:3 d_loss:0.5546 g_loss:0.0643
epoch:4 d_loss:0.5588 g_loss:0.0835
epoch:5 d_loss:0.5060 g_loss:0.1893
epoch:6 d_loss:0.4533 g_loss:0.3893
epoch:7 d_loss:0.4472 g_loss:0.5940
epoch:8 d_loss:0.4652 g_loss:0.7109
epoch:9 d_loss:0.4693 g_loss:0.7122
epoch:10 d_loss:0.4886 g_loss:0.6982
epoch:11 d_loss:0.5518 g_loss:0.5707
epoch:12 d_loss:0.5521 g_loss:0.4856
epoch:13 d_loss:0.5847 g_loss:0.3735
epoch:14 d_loss:0.4539 g_loss:0.3405
epoch:15 d_loss:0.3390 g_loss:0.3264
epoch:16 d_loss:0.2637 g_loss:0.2183
epoch:17 d_loss:0.2160 g_loss:0.1888
epoch:18 d_loss:0.1723 g_loss:0.1366
epoch:19 d_loss:0.2037 g_loss:0.3456
epoch:20 d_loss:0.2259 g_loss:0.4435
epoch:21 d_loss:0.3058 g_loss:0.3353
epoch:22 d_loss:0.3863 g_loss:0.2619
epoch:23 d_loss:0.7896 g_loss:0.1814
epoch:24 d_loss:0.8201 g_loss:0.4754
epoch:25 d_loss:0.7957 g_loss:0.7230
epoch:26 d_loss:1.1142 g_loss:0.3030
epoch:27 d_

C:\Users\bit\anaconda3\envs\deeprun\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
